In [1]:
!pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 364.4/364.4 kB 24.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.5/233.5 kB 23.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 8.8 MB/s eta 0:00:00


In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import cross_validate, train_test_split
from sklearn.metrics import accuracy_score, matthews_corrcoef, make_scorer
import optuna
from deeptlf import DeepTFL

In [3]:
# This is a function that loads training and testing data for 10-fold cross validation
def prepare_train_test_dataset(datasets, test_idx):
  test_df = datasets[test_idx]
  train_df = pd.concat([datasets[i] for i in range(len(datasets)) if i != test_idx])

  X_train = train_df.iloc[:, 1:-1].values
  X_test = test_df.iloc[:, 1:-1].values
  y_train = train_df.iloc[:, -1].values
  y_test = test_df.iloc[:, -1].values

  return X_train, X_test, y_train, y_test

In [4]:
import torch
import time

def train_with_base_deeptlf(datasets, key):
  start_time = time.time()
  print("Currently training base DeepTLF model with dataset key:", key)

  # Create a Pandas DataFrame to store all experiment results
  results_df = pd.DataFrame(columns=['Fold', 'Accuracy', 'MCC'])

  # Perform 10-fold cross validation
  dataset_list = datasets[key]

  for i in range(10):
    X_train, X_test, y_train, y_test = prepare_train_test_dataset(dataset_list, i)

    # Train the model
    model = DeepTFL()
    model.fit(X_train=X_train, y_train=y_train, X_val=X_test, y_val=y_test)

    # Evaluate the model
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    mcc = matthews_corrcoef(y_test, y_pred)

    print("Fold", i+1, "Accuracy:", accuracy, "MCC:", mcc)
    results_df.loc[i] = [i+1, accuracy, mcc]

  elapsed_time = time.time() - start_time
  print(f"Elapsed time: {elapsed_time:.2f} seconds")

  return results_df

In [5]:
def create_objective_for_deeptlf(X_train, X_test, y_train, y_test):
  def objective(trial):
      params = {
          "n_est": trial.suggest_int("n_est", 10, 50, step=5),
          "max_depth": trial.suggest_int("max_depth", 3, 10),
          "drop": trial.suggest_float("drop", 0.1, 0.5, step=0.05),
          "hidden_dim": trial.suggest_int("hidden_dim", 64, 512, step=64),
          "n_layers": trial.suggest_int("n_layers", 2, 6),
          "xgb_lr": trial.suggest_float("xgb_lr", 0.01, 0.5, step=0.01),
      }

      model = DeepTFL(
          n_est=params["n_est"],
          max_depth=params["max_depth"],
          drop=params["drop"],
          hidden_dim=params["hidden_dim"],
          n_layers=params["n_layers"],
          xgb_lr=params["xgb_lr"],
          task="class"
      )
      model.fit(X_train=X_train, y_train=y_train, X_val=X_test, y_val=y_test)

      y_pred = model.predict(X_test)
      mcc = matthews_corrcoef(y_test, y_pred)

      # Return the accuracy
      return mcc

  return objective

In [6]:
def train_with_best_hyperparameters(datasets, key):
  start_time = time.time()
  print("Currently finetuning DeepTLF model with dataset key:", key)

  # Create a Pandas DataFrame to store all experiment results
  results_df = pd.DataFrame(columns=['Fold', 'Accuracy', 'MCC'])

  # Perform 10-fold cross validation
  dataset_list = datasets[key]

  # Use the first data as the test for the HPO
  X_train, X_test, y_train, y_test = prepare_train_test_dataset(dataset_list, 0)

  objective_function = create_objective_for_deeptlf(X_train, X_test, y_train, y_test)

  study = optuna.create_study(direction='maximize')
  study.optimize(objective_function, n_trials=15, gc_after_trial=True)

  best_params = study.best_params
  print("Best hyperparameters:", best_params)

  # Use best hyperparams to conduct a 10-fold cross validation
  for i in range(10):
    X_train, X_test, y_train, y_test = prepare_train_test_dataset(dataset_list, i)

    model = DeepTFL(
        n_est=best_params["n_est"],
        max_depth=best_params["max_depth"],
        drop=best_params["drop"],
        hidden_dim=best_params["hidden_dim"],
        n_layers=best_params["n_layers"],
        xgb_lr=best_params["xgb_lr"],
        task="class"
    )
    model.fit(X_train=X_train, y_train=y_train, X_val=X_test, y_val=y_test)

    # Evaluate the model
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    mcc = matthews_corrcoef(y_test, y_pred)

    print("Fold", i+1, "Accuracy:", accuracy, "MCC:", mcc)
    results_df.loc[i] = [i+1, accuracy, mcc]

  elapsed_time = time.time() - start_time
  print(f"Elapsed time: {elapsed_time:.2f} seconds")

  return results_df

In [7]:
# Load the datasets

datasets = {}

# All

II_all_nopcc_1 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/II/all/cv/nopcc/fold_1.csv')
II_all_nopcc_2 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/II/all/cv/nopcc/fold_2.csv')
II_all_nopcc_3 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/II/all/cv/nopcc/fold_3.csv')
II_all_nopcc_4 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/II/all/cv/nopcc/fold_4.csv')
II_all_nopcc_5 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/II/all/cv/nopcc/fold_5.csv')
II_all_nopcc_6 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/II/all/cv/nopcc/fold_6.csv')
II_all_nopcc_7 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/II/all/cv/nopcc/fold_7.csv')
II_all_nopcc_8 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/II/all/cv/nopcc/fold_8.csv')
II_all_nopcc_9 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/II/all/cv/nopcc/fold_9.csv')
II_all_nopcc_10 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/II/all/cv/nopcc/fold_10.csv')
datasets['II_all_nopcc'] = [II_all_nopcc_1, II_all_nopcc_2, II_all_nopcc_3, II_all_nopcc_4, II_all_nopcc_5,
                                       II_all_nopcc_6, II_all_nopcc_7, II_all_nopcc_8, II_all_nopcc_9, II_all_nopcc_10]

II_all_pcc95_1 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/II/all/cv/pcc95/fold_1.csv')
II_all_pcc95_2 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/II/all/cv/pcc95/fold_2.csv')
II_all_pcc95_3 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/II/all/cv/pcc95/fold_3.csv')
II_all_pcc95_4 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/II/all/cv/pcc95/fold_4.csv')
II_all_pcc95_5 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/II/all/cv/pcc95/fold_5.csv')
II_all_pcc95_6 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/II/all/cv/pcc95/fold_6.csv')
II_all_pcc95_7 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/II/all/cv/pcc95/fold_7.csv')
II_all_pcc95_8 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/II/all/cv/pcc95/fold_8.csv')
II_all_pcc95_9 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/II/all/cv/pcc95/fold_9.csv')
II_all_pcc95_10 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/II/all/cv/pcc95/fold_10.csv')
datasets['II_all_pcc95'] = [II_all_pcc95_1, II_all_pcc95_2, II_all_pcc95_3, II_all_pcc95_4, II_all_pcc95_5,
                                       II_all_pcc95_6, II_all_pcc95_7, II_all_pcc95_8, II_all_pcc95_9, II_all_pcc95_10]

II_all_pcc75_1 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/II/all/cv/pcc75/fold_1.csv')
II_all_pcc75_2 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/II/all/cv/pcc75/fold_2.csv')
II_all_pcc75_3 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/II/all/cv/pcc75/fold_3.csv')
II_all_pcc75_4 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/II/all/cv/pcc75/fold_4.csv')
II_all_pcc75_5 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/II/all/cv/pcc75/fold_5.csv')
II_all_pcc75_6 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/II/all/cv/pcc75/fold_6.csv')
II_all_pcc75_7 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/II/all/cv/pcc75/fold_7.csv')
II_all_pcc75_8 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/II/all/cv/pcc75/fold_8.csv')
II_all_pcc75_9 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/II/all/cv/pcc75/fold_9.csv')
II_all_pcc75_10 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/II/all/cv/pcc75/fold_10.csv')
datasets['II_all_pcc75'] = [II_all_pcc75_1, II_all_pcc75_2, II_all_pcc75_3, II_all_pcc75_4, II_all_pcc75_5,
                                       II_all_pcc75_6, II_all_pcc75_7, II_all_pcc75_8, II_all_pcc75_9, II_all_pcc75_10]

# 550

II_550_nopcc_1 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/II/550/cv/nopcc/fold_1.csv')
II_550_nopcc_2 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/II/550/cv/nopcc/fold_2.csv')
II_550_nopcc_3 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/II/550/cv/nopcc/fold_3.csv')
II_550_nopcc_4 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/II/550/cv/nopcc/fold_4.csv')
II_550_nopcc_5 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/II/550/cv/nopcc/fold_5.csv')
II_550_nopcc_6 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/II/550/cv/nopcc/fold_6.csv')
II_550_nopcc_7 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/II/550/cv/nopcc/fold_7.csv')
II_550_nopcc_8 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/II/550/cv/nopcc/fold_8.csv')
II_550_nopcc_9 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/II/550/cv/nopcc/fold_9.csv')
II_550_nopcc_10 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/II/550/cv/nopcc/fold_10.csv')
datasets['II_550_nopcc'] = [II_550_nopcc_1, II_550_nopcc_2, II_550_nopcc_3, II_550_nopcc_4, II_550_nopcc_5,
                                       II_550_nopcc_6, II_550_nopcc_7, II_550_nopcc_8, II_550_nopcc_9, II_550_nopcc_10]

II_550_pcc95_1 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/II/550/cv/pcc95/fold_1.csv')
II_550_pcc95_2 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/II/550/cv/pcc95/fold_2.csv')
II_550_pcc95_3 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/II/550/cv/pcc95/fold_3.csv')
II_550_pcc95_4 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/II/550/cv/pcc95/fold_4.csv')
II_550_pcc95_5 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/II/550/cv/pcc95/fold_5.csv')
II_550_pcc95_6 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/II/550/cv/pcc95/fold_6.csv')
II_550_pcc95_7 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/II/550/cv/pcc95/fold_7.csv')
II_550_pcc95_8 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/II/550/cv/pcc95/fold_8.csv')
II_550_pcc95_9 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/II/550/cv/pcc95/fold_9.csv')
II_550_pcc95_10 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/II/550/cv/pcc95/fold_10.csv')
datasets['II_550_pcc95'] = [II_550_pcc95_1, II_550_pcc95_2, II_550_pcc95_3, II_550_pcc95_4, II_550_pcc95_5,
                                       II_550_pcc95_6, II_550_pcc95_7, II_550_pcc95_8, II_550_pcc95_9, II_550_pcc95_10]

II_550_pcc75_1 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/II/550/cv/pcc75/fold_1.csv')
II_550_pcc75_2 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/II/550/cv/pcc75/fold_2.csv')
II_550_pcc75_3 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/II/550/cv/pcc75/fold_3.csv')
II_550_pcc75_4 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/II/550/cv/pcc75/fold_4.csv')
II_550_pcc75_5 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/II/550/cv/pcc75/fold_5.csv')
II_550_pcc75_6 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/II/550/cv/pcc75/fold_6.csv')
II_550_pcc75_7 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/II/550/cv/pcc75/fold_7.csv')
II_550_pcc75_8 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/II/550/cv/pcc75/fold_8.csv')
II_550_pcc75_9 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/II/550/cv/pcc75/fold_9.csv')
II_550_pcc75_10 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/II/550/cv/pcc75/fold_10.csv')
datasets['II_550_pcc75'] = [II_550_pcc75_1, II_550_pcc75_2, II_550_pcc75_3, II_550_pcc75_4, II_550_pcc75_5,
                                       II_550_pcc75_6, II_550_pcc75_7, II_550_pcc75_8, II_550_pcc75_9, II_550_pcc75_10]

# 450

II_450_nopcc_1 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/II/450/cv/nopcc/fold_1.csv')
II_450_nopcc_2 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/II/450/cv/nopcc/fold_2.csv')
II_450_nopcc_3 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/II/450/cv/nopcc/fold_3.csv')
II_450_nopcc_4 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/II/450/cv/nopcc/fold_4.csv')
II_450_nopcc_5 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/II/450/cv/nopcc/fold_5.csv')
II_450_nopcc_6 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/II/450/cv/nopcc/fold_6.csv')
II_450_nopcc_7 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/II/450/cv/nopcc/fold_7.csv')
II_450_nopcc_8 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/II/450/cv/nopcc/fold_8.csv')
II_450_nopcc_9 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/II/450/cv/nopcc/fold_9.csv')
II_450_nopcc_10 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/II/450/cv/nopcc/fold_10.csv')
datasets['II_450_nopcc'] = [II_450_nopcc_1, II_450_nopcc_2, II_450_nopcc_3, II_450_nopcc_4, II_450_nopcc_5,
                                       II_450_nopcc_6, II_450_nopcc_7, II_450_nopcc_8, II_450_nopcc_9, II_450_nopcc_10]

II_450_pcc95_1 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/II/450/cv/pcc95/fold_1.csv')
II_450_pcc95_2 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/II/450/cv/pcc95/fold_2.csv')
II_450_pcc95_3 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/II/450/cv/pcc95/fold_3.csv')
II_450_pcc95_4 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/II/450/cv/pcc95/fold_4.csv')
II_450_pcc95_5 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/II/450/cv/pcc95/fold_5.csv')
II_450_pcc95_6 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/II/450/cv/pcc95/fold_6.csv')
II_450_pcc95_7 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/II/450/cv/pcc95/fold_7.csv')
II_450_pcc95_8 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/II/450/cv/pcc95/fold_8.csv')
II_450_pcc95_9 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/II/450/cv/pcc95/fold_9.csv')
II_450_pcc95_10 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/II/450/cv/pcc95/fold_10.csv')
datasets['II_450_pcc95'] = [II_450_pcc95_1, II_450_pcc95_2, II_450_pcc95_3, II_450_pcc95_4, II_450_pcc95_5,
                                       II_450_pcc95_6, II_450_pcc95_7, II_450_pcc95_8, II_450_pcc95_9, II_450_pcc95_10]

II_450_pcc75_1 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/II/450/cv/pcc75/fold_1.csv')
II_450_pcc75_2 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/II/450/cv/pcc75/fold_2.csv')
II_450_pcc75_3 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/II/450/cv/pcc75/fold_3.csv')
II_450_pcc75_4 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/II/450/cv/pcc75/fold_4.csv')
II_450_pcc75_5 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/II/450/cv/pcc75/fold_5.csv')
II_450_pcc75_6 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/II/450/cv/pcc75/fold_6.csv')
II_450_pcc75_7 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/II/450/cv/pcc75/fold_7.csv')
II_450_pcc75_8 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/II/450/cv/pcc75/fold_8.csv')
II_450_pcc75_9 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/II/450/cv/pcc75/fold_9.csv')
II_450_pcc75_10 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/II/450/cv/pcc75/fold_10.csv')
datasets['II_450_pcc75'] = [II_450_pcc75_1, II_450_pcc75_2, II_450_pcc75_3, II_450_pcc75_4, II_450_pcc75_5,
                                       II_450_pcc75_6, II_450_pcc75_7, II_450_pcc75_8, II_450_pcc75_9, II_450_pcc75_10]

# 350

II_350_nopcc_1 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/II/350/cv/nopcc/fold_1.csv')
II_350_nopcc_2 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/II/350/cv/nopcc/fold_2.csv')
II_350_nopcc_3 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/II/350/cv/nopcc/fold_3.csv')
II_350_nopcc_4 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/II/350/cv/nopcc/fold_4.csv')
II_350_nopcc_5 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/II/350/cv/nopcc/fold_5.csv')
II_350_nopcc_6 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/II/350/cv/nopcc/fold_6.csv')
II_350_nopcc_7 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/II/350/cv/nopcc/fold_7.csv')
II_350_nopcc_8 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/II/350/cv/nopcc/fold_8.csv')
II_350_nopcc_9 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/II/350/cv/nopcc/fold_9.csv')
II_350_nopcc_10 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/II/350/cv/nopcc/fold_10.csv')
datasets['II_350_nopcc'] = [II_350_nopcc_1, II_350_nopcc_2, II_350_nopcc_3, II_350_nopcc_4, II_350_nopcc_5,
                                       II_350_nopcc_6, II_350_nopcc_7, II_350_nopcc_8, II_350_nopcc_9, II_350_nopcc_10]

II_350_pcc95_1 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/II/350/cv/pcc95/fold_1.csv')
II_350_pcc95_2 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/II/350/cv/pcc95/fold_2.csv')
II_350_pcc95_3 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/II/350/cv/pcc95/fold_3.csv')
II_350_pcc95_4 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/II/350/cv/pcc95/fold_4.csv')
II_350_pcc95_5 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/II/350/cv/pcc95/fold_5.csv')
II_350_pcc95_6 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/II/350/cv/pcc95/fold_6.csv')
II_350_pcc95_7 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/II/350/cv/pcc95/fold_7.csv')
II_350_pcc95_8 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/II/350/cv/pcc95/fold_8.csv')
II_350_pcc95_9 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/II/350/cv/pcc95/fold_9.csv')
II_350_pcc95_10 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/II/350/cv/pcc95/fold_10.csv')
datasets['II_350_pcc95'] = [II_350_pcc95_1, II_350_pcc95_2, II_350_pcc95_3, II_350_pcc95_4, II_350_pcc95_5,
                                       II_350_pcc95_6, II_350_pcc95_7, II_350_pcc95_8, II_350_pcc95_9, II_350_pcc95_10]

II_350_pcc75_1 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/II/350/cv/pcc75/fold_1.csv')
II_350_pcc75_2 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/II/350/cv/pcc75/fold_2.csv')
II_350_pcc75_3 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/II/350/cv/pcc75/fold_3.csv')
II_350_pcc75_4 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/II/350/cv/pcc75/fold_4.csv')
II_350_pcc75_5 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/II/350/cv/pcc75/fold_5.csv')
II_350_pcc75_6 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/II/350/cv/pcc75/fold_6.csv')
II_350_pcc75_7 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/II/350/cv/pcc75/fold_7.csv')
II_350_pcc75_8 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/II/350/cv/pcc75/fold_8.csv')
II_350_pcc75_9 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/II/350/cv/pcc75/fold_9.csv')
II_350_pcc75_10 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/II/350/cv/pcc75/fold_10.csv')
datasets['II_350_pcc75'] = [II_350_pcc75_1, II_350_pcc75_2, II_350_pcc75_3, II_350_pcc75_4, II_350_pcc75_5,
                                       II_350_pcc75_6, II_350_pcc75_7, II_350_pcc75_8, II_350_pcc75_9, II_350_pcc75_10]


In [8]:
def view_results_df(results_df, key):
    print("10-fold cross validation results for results_df with key:", key)
    for i in range(len(results_df)):
        # Format accuracy and MCC to 3 decimal places
        accuracy = f"{results_df['Accuracy'][i]:.3f}"
        mcc = f"{results_df['MCC'][i]:.3f}"
        print(f"Fold {i}, Accuracy: {accuracy}, MCC: {mcc}")


In [9]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [10]:
II_all_nopcc_results_df = train_with_base_deeptlf(datasets, 'II_all_nopcc')
print("\n---------------------------------------------------------------------\n")

Currently training base DeepTLF model with dataset key: II_all_nopcc


Epochs:  51%|█████     | 51/100 [00:16<00:15,  3.17it/s]

Early stopping
Fold 1 Accuracy: 0.8455284552845529 MCC: 0.6925232487431626



Epochs:  24%|██▍       | 24/100 [00:07<00:23,  3.20it/s]

Early stopping
Fold 2 Accuracy: 0.7877551020408163 MCC: 0.5757536622893442


...

Early stopping
Fold 8 Accuracy: 0.8081632653061225 MCC: 0.6208048013719194



Epochs:  33%|███▎      | 33/100 [00:10<00:21,  3.14it/s]

Early stopping
Fold 9 Accuracy: 0.8653061224489796 MCC: 0.7362775909738983



Epochs:  32%|███▏      | 32/100 [00:10<00:23,  2.93it/s]

Early stopping
Fold 10 Accuracy: 0.8448979591836735 MCC: 0.6993690645690265
Elapsed time: 134.86 seconds

---------------------------------------------------------------------



In [11]:
II_all_pcc95_results_df = train_with_base_deeptlf(datasets, 'II_all_pcc95')
print("\n---------------------------------------------------------------------\n")

Currently training base DeepTLF model with dataset key: II_all_pcc95


Epochs:  24%|██▍       | 24/100 [00:03<00:09,  7.68it/s]


Early stopping
Fold 1 Accuracy: 0.7886178861788617 MCC: 0.58351784605382


Epochs:  31%|███       | 31/100 [00:04<00:09,  7.61it/s]


Early stopping
Fold 2 Accuracy: 0.8285714285714286 MCC: 0.6698660401866047


...

Early stopping
Fold 8 Accuracy: 0.8571428571428571 MCC: 0.7170752702804015


Epochs:  25%|██▌       | 25/100 [00:03<00:10,  7.24it/s]


Early stopping
Fold 9 Accuracy: 0.8693877551020408 MCC: 0.740420575383617


Epochs:  31%|███       | 31/100 [00:04<00:10,  6.50it/s]


Early stopping
Fold 10 Accuracy: 0.8571428571428571 MCC: 0.7153838111123045
Elapsed time: 59.36 seconds

---------------------------------------------------------------------



In [12]:
II_all_pcc75_results_df = train_with_base_deeptlf(datasets, 'II_all_pcc75')
print("\n---------------------------------------------------------------------\n")

Currently training base DeepTLF model with dataset key: II_all_pcc75


Epochs:  28%|██▊       | 28/100 [00:02<00:06, 11.51it/s]


Early stopping
Fold 1 Accuracy: 0.7926829268292683 MCC: 0.5869391856534222


Epochs:  26%|██▌       | 26/100 [00:02<00:06, 11.40it/s]


Early stopping
Fold 2 Accuracy: 0.8 MCC: 0.6002133026746642


...

Early stopping
Fold 8 Accuracy: 0.8081632653061225 MCC: 0.616897976585444


Epochs:  25%|██▌       | 25/100 [00:02<00:07, 10.34it/s]


Early stopping
Fold 9 Accuracy: 0.8244897959183674 MCC: 0.6528090185632586


Epochs:  29%|██▉       | 29/100 [00:02<00:06, 10.89it/s]


Early stopping
Fold 10 Accuracy: 0.8489795918367347 MCC: 0.6979673457685988
Elapsed time: 33.24 seconds

---------------------------------------------------------------------



In [13]:
II_550_nopcc_results_df = train_with_base_deeptlf(datasets, 'II_550_nopcc')
print("\n---------------------------------------------------------------------\n")

Currently training base DeepTLF model with dataset key: II_550_nopcc


Epochs:  41%|████      | 41/100 [00:05<00:07,  7.39it/s]


Early stopping
Fold 1 Accuracy: 0.9 MCC: 0.8033264176742436


Epochs:  29%|██▉       | 29/100 [00:04<00:10,  7.09it/s]


Early stopping
Fold 2 Accuracy: 0.8909090909090909 MCC: 0.7902222465352922


...

Early stopping
Fold 8 Accuracy: 0.9 MCC: 0.806559132617443


Epochs:  57%|█████▋    | 57/100 [00:08<00:06,  6.93it/s]


Early stopping
Fold 9 Accuracy: 0.9090909090909091 MCC: 0.8181818181818182


Epochs:  29%|██▉       | 29/100 [00:03<00:09,  7.25it/s]


Early stopping
Fold 10 Accuracy: 0.8807339449541285 MCC: 0.7624846931439233
Elapsed time: 75.71 seconds

---------------------------------------------------------------------



In [14]:

II_550_pcc95_results_df = train_with_base_deeptlf(datasets, 'II_550_pcc95')
print("\n---------------------------------------------------------------------\n")

Currently training base DeepTLF model with dataset key: II_550_pcc95


Epochs:  29%|██▉       | 29/100 [00:02<00:04, 14.38it/s]


Early stopping
Fold 1 Accuracy: 0.9090909090909091 MCC: 0.8230942281400455


Epochs:  35%|███▌      | 35/100 [00:02<00:04, 16.20it/s]


Early stopping
Fold 2 Accuracy: 0.9272727272727272 MCC: 0.8596761938351586


...

Early stopping
Fold 8 Accuracy: 0.9181818181818182 MCC: 0.8376105968386142


Epochs:  38%|███▊      | 38/100 [00:02<00:04, 14.16it/s]


Early stopping
Fold 9 Accuracy: 0.8636363636363636 MCC: 0.7283570407292297


Epochs:  31%|███       | 31/100 [00:01<00:04, 16.34it/s]

Early stopping
Fold 10 Accuracy: 0.8990825688073395 MCC: 0.8045911653027329
Elapsed time: 37.58 seconds

---------------------------------------------------------------------



In [15]:

II_550_pcc75_results_df = train_with_base_deeptlf(datasets, 'II_550_pcc75')
print("\n---------------------------------------------------------------------\n")

Currently training base DeepTLF model with dataset key: II_550_pcc75


Epochs:  36%|███▌      | 36/100 [00:01<00:02, 23.37it/s]


Early stopping
Fold 1 Accuracy: 0.8636363636363636 MCC: 0.7332355751067664


Epochs:  45%|████▌     | 45/100 [00:02<00:02, 22.33it/s]


Early stopping
Fold 2 Accuracy: 0.9181818181818182 MCC: 0.8365019125713041


...

Early stopping
Fold 8 Accuracy: 0.8545454545454545 MCC: 0.7265959585039445


Epochs:  49%|████▉     | 49/100 [00:02<00:02, 21.28it/s]


Early stopping
Fold 9 Accuracy: 0.8909090909090909 MCC: 0.7838940385932954


Epochs:  60%|██████    | 60/100 [00:02<00:01, 22.28it/s]

Early stopping
Fold 10 Accuracy: 0.944954128440367 MCC: 0.8898989898989899
Elapsed time: 23.33 seconds

---------------------------------------------------------------------



In [16]:
II_450_nopcc_results_df = train_with_base_deeptlf(datasets, 'II_450_nopcc')
print("\n---------------------------------------------------------------------\n")

Currently training base DeepTLF model with dataset key: II_450_nopcc


Epochs:  29%|██▉       | 29/100 [00:03<00:07,  9.59it/s]


Early stopping
Fold 1 Accuracy: 0.8888888888888888 MCC: 0.7903678163100035


Epochs:  29%|██▉       | 29/100 [00:03<00:07,  9.28it/s]


Early stopping
Fold 2 Accuracy: 0.8555555555555555 MCC: 0.719874033065356


...

Early stopping
Fold 8 Accuracy: 0.8666666666666667 MCC: 0.734058687945213


Epochs:  37%|███▋      | 37/100 [00:04<00:06,  9.23it/s]


Early stopping
Fold 9 Accuracy: 0.9666666666666667 MCC: 0.9335638713962128


Epochs:  29%|██▉       | 29/100 [00:03<00:07,  9.37it/s]


Early stopping
Fold 10 Accuracy: 0.8876404494382022 MCC: 0.7752525252525253
Elapsed time: 55.74 seconds

---------------------------------------------------------------------



In [17]:

II_450_pcc95_results_df = train_with_base_deeptlf(datasets, 'II_450_pcc95')
print("\n---------------------------------------------------------------------\n")

Currently training base DeepTLF model with dataset key: II_450_pcc95


Epochs:  34%|███▍      | 34/100 [00:01<00:03, 17.36it/s]


Early stopping
Fold 1 Accuracy: 0.9666666666666667 MCC: 0.9354143466934853


Epochs:  27%|██▋       | 27/100 [00:01<00:04, 17.93it/s]


Early stopping
Fold 2 Accuracy: 0.8888888888888888 MCC: 0.7903678163100035


...

Early stopping
Fold 8 Accuracy: 0.8444444444444444 MCC: 0.6888888888888889


Epochs:  43%|████▎     | 43/100 [00:02<00:03, 17.66it/s]


Early stopping
Fold 9 Accuracy: 0.9444444444444444 MCC: 0.889108448948774


Epochs:  28%|██▊       | 28/100 [00:01<00:04, 17.79it/s]


Early stopping
Fold 10 Accuracy: 0.9213483146067416 MCC: 0.8428500318082266
Elapsed time: 31.21 seconds

---------------------------------------------------------------------



In [18]:

II_450_pcc75_results_df = train_with_base_deeptlf(datasets, 'II_450_pcc75')
print("\n---------------------------------------------------------------------\n")

Currently training base DeepTLF model with dataset key: II_450_pcc75


Epochs:  36%|███▌      | 36/100 [00:01<00:02, 25.77it/s]


Early stopping
Fold 1 Accuracy: 0.9666666666666667 MCC: 0.9335638713962128


Epochs:  31%|███       | 31/100 [00:01<00:02, 27.50it/s]


Early stopping
Fold 2 Accuracy: 0.9111111111111111 MCC: 0.8230354986052387


...

Early stopping
Fold 8 Accuracy: 0.8777777777777778 MCC: 0.755742181606458


Epochs:  29%|██▉       | 29/100 [00:01<00:02, 27.09it/s]


Early stopping
Fold 9 Accuracy: 0.8777777777777778 MCC: 0.7602631123499285


Epochs:  45%|████▌     | 45/100 [00:01<00:01, 29.56it/s]


Early stopping
Fold 10 Accuracy: 0.9101123595505618 MCC: 0.8237120116864876
Elapsed time: 18.31 seconds

---------------------------------------------------------------------



In [19]:
II_350_nopcc_results_df = train_with_base_deeptlf(datasets, 'II_350_nopcc')
print("\n---------------------------------------------------------------------\n")


Currently training base DeepTLF model with dataset key: II_350_nopcc


Epochs:  35%|███▌      | 35/100 [00:03<00:05, 11.02it/s]


Early stopping
Fold 1 Accuracy: 0.8857142857142857 MCC: 0.7765163665331185


Epochs:  30%|███       | 30/100 [00:02<00:05, 11.72it/s]


Early stopping
Fold 2 Accuracy: 0.8285714285714286 MCC: 0.6582183780869638


...

Early stopping
Fold 8 Accuracy: 0.9 MCC: 0.8003267306650412


Epochs:  40%|████      | 40/100 [00:03<00:04, 12.39it/s]


Early stopping
Fold 9 Accuracy: 1.0 MCC: 1.0


Epochs:  30%|███       | 30/100 [00:02<00:05, 12.04it/s]


Early stopping
Fold 10 Accuracy: 0.855072463768116 MCC: 0.7100840336134454
Elapsed time: 52.33 seconds

---------------------------------------------------------------------



In [20]:
II_350_pcc95_results_df = train_with_base_deeptlf(datasets, 'II_350_pcc95')
print("\n---------------------------------------------------------------------\n")


Currently training base DeepTLF model with dataset key: II_350_pcc95


Epochs:  47%|████▋     | 47/100 [00:02<00:02, 23.14it/s]


Early stopping
Fold 1 Accuracy: 0.9428571428571428 MCC: 0.8857142857142857


Epochs:  26%|██▌       | 26/100 [00:01<00:03, 23.33it/s]


Early stopping
Fold 2 Accuracy: 0.8142857142857143 MCC: 0.6288281455225324


...

Early stopping
Fold 8 Accuracy: 0.9428571428571428 MCC: 0.8871639008998208


Epochs:  68%|██████▊   | 68/100 [00:02<00:01, 23.91it/s]


Early stopping
Fold 9 Accuracy: 1.0 MCC: 1.0


Epochs:  36%|███▌      | 36/100 [00:01<00:02, 23.73it/s]

Early stopping
Fold 10 Accuracy: 0.9130434782608695 MCC: 0.8275864995945614
Elapsed time: 30.72 seconds

---------------------------------------------------------------------



In [21]:
II_350_pcc75_results_df = train_with_base_deeptlf(datasets, 'II_350_pcc75')

Currently training base DeepTLF model with dataset key: II_350_pcc75


Epochs:  29%|██▉       | 29/100 [00:00<00:02, 34.13it/s]


Early stopping
Fold 1 Accuracy: 0.8857142857142857 MCC: 0.7765163665331185


Epochs:  54%|█████▍    | 54/100 [00:01<00:01, 39.12it/s]


Early stopping
Fold 2 Accuracy: 0.8857142857142857 MCC: 0.7726911394933923


...

Early stopping
Fold 8 Accuracy: 0.9571428571428572 MCC: 0.9146591207600471


Epochs:  48%|████▊     | 48/100 [00:01<00:01, 35.55it/s]


Early stopping
Fold 9 Accuracy: 1.0 MCC: 1.0


Epochs:  34%|███▍      | 34/100 [00:00<00:01, 35.34it/s]

Early stopping
Fold 10 Accuracy: 0.8840579710144928 MCC: 0.7680672268907563
Elapsed time: 17.22 seconds


In [22]:
print('\n---------------------------------------------------------------------\n')
view_results_df(II_all_nopcc_results_df, 'II_all_nopcc')
print('\n---------------------------------------------------------------------\n')
view_results_df(II_all_pcc95_results_df, 'II_all_pcc95')
print('\n---------------------------------------------------------------------\n')
view_results_df(II_all_nopcc_results_df, 'II_all_pcc75')
print('\n---------------------------------------------------------------------\n')
view_results_df(II_550_nopcc_results_df, 'II_550_nopcc')
print('\n---------------------------------------------------------------------\n')
view_results_df(II_550_pcc95_results_df, 'II_550_pcc95')
print('\n---------------------------------------------------------------------\n')
view_results_df(II_550_nopcc_results_df, 'II_550_pcc75')
print('\n---------------------------------------------------------------------\n')
view_results_df(II_450_nopcc_results_df, 'II_450_nopcc')
print('\n---------------------------------------------------------------------\n')
view_results_df(II_450_pcc95_results_df, 'II_450_pcc95')
print('\n---------------------------------------------------------------------\n')
view_results_df(II_450_nopcc_results_df, 'II_450_pcc75')
print('\n---------------------------------------------------------------------\n')
view_results_df(II_350_nopcc_results_df, 'II_350_nopcc')
print('\n---------------------------------------------------------------------\n')
view_results_df(II_350_pcc95_results_df, 'II_350_pcc95')
print('\n---------------------------------------------------------------------\n')
view_results_df(II_350_nopcc_results_df, 'II_350_pcc75')
print('\n---------------------------------------------------------------------\n')



---------------------------------------------------------------------

10-fold cross validation results for results_df with key: II_all_nopcc
Fold 0, Accuracy: 0.846, MCC: 0.693
Fold 1, Accuracy: 0.788, MCC: 0.576
Fold 2, Accuracy: 0.845, MCC: 0.690
Fold 3, Accuracy: 0.865, MCC: 0.732
Fold 4, Accuracy: 0.771, MCC: 0.552
Fold 5, Accuracy: 0.796, MCC: 0.604
Fold 6, Accuracy: 0.841, MCC: 0.683
Fold 7, Accuracy: 0.808, MCC: 0.621
Fold 8, Accuracy: 0.865, MCC: 0.736
Fold 9, Accuracy: 0.845, MCC: 0.699

---------------------------------------------------------------------

10-fold cross validation results for results_df with key: II_all_pcc95
Fold 0, Accuracy: 0.789, MCC: 0.584
Fold 1, Accuracy: 0.829, MCC: 0.670
Fold 2, Accuracy: 0.910, MCC: 0.821
Fold 3, Accuracy: 0.857, MCC: 0.717
Fold 4, Accuracy: 0.796, MCC: 0.592
Fold 5, Accuracy: 0.865, MCC: 0.732
Fold 6, Accuracy: 0.849, MCC: 0.698
Fold 7, Accuracy: 0.857, MCC: 0.717
Fold 8, Accuracy: 0.869, MCC: 0.740
Fold 9, Accuracy: 0.857, MCC: 

In [23]:
II_all_nopcc_results_df = train_with_best_hyperparameters(datasets, 'II_all_nopcc')

[I 2025-01-04 18:30:26,672] A new study created in memory with name: no-name-4be507be-578a-46c6-a731-c0fb4d0cd3b3


Currently finetuning DeepTLF model with dataset key: II_all_nopcc


Epochs:  32%|███▏      | 32/100 [00:25<00:54,  1.24it/s]

Early stopping



[I 2025-01-04 18:31:04,628] Trial 0 finished with value: 0.6521546507260246 and parameters: {'n_est': 30, 'max_depth': 8, 'drop': 0.35, 'hidden_dim': 448, 'n_layers': 4, 'xgb_lr': 0.23}. Best is trial 0 with value: 0.6521546507260246.
Epochs:  46%|████▌     | 46/100 [00:50<00:59,  1.10s/it]

...

Early stopping
Fold 8 Accuracy: 0.8204081632653061 MCC: 0.6408718384471844



Epochs:  30%|███       | 30/100 [00:08<00:19,  3.66it/s]

Early stopping
Fold 9 Accuracy: 0.8489795918367347 MCC: 0.6991665190551163



Epochs:  28%|██▊       | 28/100 [00:09<00:23,  3.05it/s]

Early stopping
Fold 10 Accuracy: 0.8367346938775511 MCC: 0.6737798243886418
Elapsed time: 542.17 seconds


In [24]:
II_all_pcc95_results_df = train_with_best_hyperparameters(datasets, 'II_all_pcc95')

[I 2025-01-04 18:39:28,851] A new study created in memory with name: no-name-5c851517-b5df-41cb-8389-25fb928f2aed


Currently finetuning DeepTLF model with dataset key: II_all_pcc95


Epochs:  26%|██▌       | 26/100 [00:03<00:09,  7.79it/s]
[I 2025-01-04 18:39:34,082] Trial 0 finished with value: 0.6260990336999411 and parameters: {'n_est': 20, 'max_depth': 4, 'drop': 0.2, 'hidden_dim': 384, 'n_layers': 3, 'xgb_lr': 0.44}. Best is trial 0 with value: 0.6260990336999411.


Early stopping


Epochs:  24%|██▍       | 24/100 [00:16<00:52,  1.44it/s]

...

Early stopping
Fold 8 Accuracy: 0.9020408163265307 MCC: 0.805005795491254


Epochs:  31%|███       | 31/100 [00:06<00:13,  4.96it/s]

Early stopping
Fold 9 Accuracy: 0.8530612244897959 MCC: 0.7062312578162744



Epochs:  34%|███▍      | 34/100 [00:07<00:14,  4.48it/s]

Early stopping
Fold 10 Accuracy: 0.8408163265306122 MCC: 0.6818000545440065
Elapsed time: 371.34 seconds


In [25]:
II_all_pcc75_results_df = train_with_best_hyperparameters(datasets, 'II_all_pcc75')

[I 2025-01-04 18:45:40,197] A new study created in memory with name: no-name-35d47d70-c428-403d-bc2d-9c40b5a4d631


Currently finetuning DeepTLF model with dataset key: II_all_pcc75


Epochs:  23%|██▎       | 23/100 [00:12<00:43,  1.78it/s]

Early stopping



[I 2025-01-04 18:45:58,098] Trial 0 finished with value: 0.5367450401216932 and parameters: {'n_est': 35, 'max_depth': 9, 'drop': 0.15000000000000002, 'hidden_dim': 512, 'n_layers': 6, 'xgb_lr': 0.48000000000000004}. Best is trial 0 with value: 0.5367450401216932.
Epochs:  68%|██████▊   | 68/100 [00:06<00:03, 10.06it/s]
[I 2025-01-04 18:46:06,139] Trial 1 finished with value: 0.704600036011158 and parameters: {'n_est': 20, 'max_depth': 6, 'drop': 0.45000000000000007, 'hidden_dim': 64, 'n_layers': 3, 'xgb_lr': 0.04}. Best is trial 1 with value: 0.704600036011158.


...

Early stopping
Fold 8 Accuracy: 0.8367346938775511 MCC: 0.6749911647545869


Epochs:  40%|████      | 40/100 [00:03<00:05, 10.69it/s]


Early stopping
Fold 9 Accuracy: 0.8653061224489796 MCC: 0.7318659171974287


Epochs:  45%|████▌     | 45/100 [00:04<00:05, 10.15it/s]


Early stopping
Fold 10 Accuracy: 0.8816326530612245 MCC: 0.7674966854845937
Elapsed time: 223.46 seconds


In [26]:
II_550_nopcc_results_df = train_with_best_hyperparameters(datasets, 'II_550_nopcc')

[I 2025-01-04 18:49:23,668] A new study created in memory with name: no-name-273552b4-717e-418c-bba8-81ed01eaf471


Currently finetuning DeepTLF model with dataset key: II_550_nopcc


Epochs:  40%|████      | 40/100 [00:02<00:04, 14.83it/s]
[I 2025-01-04 18:49:28,098] Trial 0 finished with value: 0.727392967453308 and parameters: {'n_est': 10, 'max_depth': 7, 'drop': 0.4, 'hidden_dim': 384, 'n_layers': 3, 'xgb_lr': 0.32}. Best is trial 0 with value: 0.727392967453308.


Early stopping


Epochs:  71%|███████   | 71/100 [00:19<00:07,  3.63it/s]

...

Early stopping
Fold 8 Accuracy: 0.9 MCC: 0.8011927448021527


Epochs:  26%|██▌       | 26/100 [00:03<00:10,  7.06it/s]


Early stopping
Fold 9 Accuracy: 0.8363636363636363 MCC: 0.6799586772512979


Epochs:  44%|████▍     | 44/100 [00:05<00:07,  7.98it/s]


Early stopping
Fold 10 Accuracy: 0.8899082568807339 MCC: 0.7886149148721336
Elapsed time: 294.82 seconds


In [27]:
II_550_pcc95_results_df = train_with_best_hyperparameters(datasets, 'II_550_pcc95')

[I 2025-01-04 18:54:18,492] A new study created in memory with name: no-name-57f2687a-fb29-41e0-abe5-4797f92d98e1


Currently finetuning DeepTLF model with dataset key: II_550_pcc95


Epochs:  60%|██████    | 60/100 [00:12<00:08,  4.74it/s]


Early stopping


[I 2025-01-04 18:54:36,277] Trial 0 finished with value: 0.8001322641986387 and parameters: {'n_est': 50, 'max_depth': 8, 'drop': 0.30000000000000004, 'hidden_dim': 320, 'n_layers': 6, 'xgb_lr': 0.02}. Best is trial 0 with value: 0.8001322641986387.
Epochs:  45%|████▌     | 45/100 [00:02<00:02, 19.97it/s]
[I 2025-01-04 18:54:39,389] Trial 1 finished with value: 0.7474338507517468 and parameters: {'n_est': 15, 'max_depth': 3, 'drop': 0.4, 'hidden_dim': 256, 'n_layers': 6, 'xgb_lr': 0.06999999999999999}. Best is trial 0 with value: 0.8001322641986387.


...

Early stopping


Fold 9 Accuracy: 0.8909090909090909 MCC: 0.7823355995993853


Epochs:  44%|████▍     | 44/100 [00:14<00:18,  3.07it/s]

Early stopping


Fold 10 Accuracy: 0.9357798165137615 MCC: 0.8717171717171717
Elapsed time: 366.03 seconds


In [28]:
II_550_pcc75_results_df = train_with_best_hyperparameters(datasets, 'II_550_pcc75')

[I 2025-01-04 19:00:24,537] A new study created in memory with name: no-name-9441d25b-86c0-477e-89ae-8815a87103d7


Currently finetuning DeepTLF model with dataset key: II_550_pcc75


Epochs:  26%|██▌       | 26/100 [00:05<00:15,  4.89it/s]
[I 2025-01-04 19:00:31,869] Trial 0 finished with value: 0.7668115805072325 and parameters: {'n_est': 30, 'max_depth': 8, 'drop': 0.15000000000000002, 'hidden_dim': 512, 'n_layers': 4, 'xgb_lr': 0.45}. Best is trial 0 with value: 0.7668115805072325.


Early stopping


Epochs:  64%|██████▍   | 64/100 [00:17<00:09,  3.73it/s]
[I 2025-01-04 19:00:51,609] Trial 1 finished with value: 0.8365019125713041 and parameters: {'n_est': 35, 'max_depth': 9, 'drop': 0.45000000000000007, 'hidden_dim': 448, 'n_layers': 3, 'xgb_lr': 0.29000000000000004}. Best is trial 1 with value: 0.8365019125713041.


...

Early stopping
Fold 8 Accuracy: 0.9 MCC: 0.8011927448021527


Epochs:  25%|██▌       | 25/100 [00:00<00:02, 26.69it/s]


Early stopping
Fold 9 Accuracy: 0.8181818181818182 MCC: 0.6401843996644798


Epochs:  29%|██▉       | 29/100 [00:01<00:02, 26.18it/s]


Early stopping
Fold 10 Accuracy: 0.8899082568807339 MCC: 0.7803974844810718
Elapsed time: 134.43 seconds


In [29]:
II_450_nopcc_results_df = train_with_best_hyperparameters(datasets, 'II_450_nopcc')

[I 2025-01-04 19:02:38,974] A new study created in memory with name: no-name-83e6bddd-35ea-4ec5-ad2a-79331186e085


Currently finetuning DeepTLF model with dataset key: II_450_nopcc


Epochs:  42%|████▏     | 42/100 [00:01<00:02, 26.69it/s]
[I 2025-01-04 19:02:41,879] Trial 0 finished with value: 0.8944271909999159 and parameters: {'n_est': 10, 'max_depth': 6, 'drop': 0.30000000000000004, 'hidden_dim': 256, 'n_layers': 3, 'xgb_lr': 0.15000000000000002}. Best is trial 0 with value: 0.8944271909999159.


Early stopping


Epochs:  61%|██████    | 61/100 [00:12<00:07,  4.93it/s]
[I 2025-01-04 19:02:58,673] Trial 1 finished with value: 0.889108448948774 and parameters: {'n_est': 25, 'max_depth': 10, 'drop': 0.1, 'hidden_dim': 512, 'n_layers': 5, 'xgb_lr': 0.15000000000000002}. Best is trial 0 with value: 0.8944271909999159.


...

Early stopping


Fold 9 Accuracy: 0.9111111111111111 MCC: 0.8230354986052387


Epochs:  65%|██████▌   | 65/100 [00:18<00:09,  3.51it/s]

Early stopping


Fold 10 Accuracy: 0.9213483146067416 MCC: 0.848234278643393
Elapsed time: 521.11 seconds


In [30]:
II_450_pcc95_results_df = train_with_best_hyperparameters(datasets, 'II_450_pcc95')

[I 2025-01-04 19:11:20,097] A new study created in memory with name: no-name-4235f413-e9e3-43b9-a1cb-572b3c5e1a8e


Currently finetuning DeepTLF model with dataset key: II_450_pcc95


Epochs:  33%|███▎      | 33/100 [00:06<00:14,  4.77it/s]
[I 2025-01-04 19:11:30,334] Trial 0 finished with value: 0.8675239039352517 and parameters: {'n_est': 30, 'max_depth': 7, 'drop': 0.2, 'hidden_dim': 448, 'n_layers': 4, 'xgb_lr': 0.41000000000000003}. Best is trial 0 with value: 0.8675239039352517.


Early stopping


Epochs:  49%|████▉     | 49/100 [00:04<00:05,  9.87it/s]
[I 2025-01-04 19:11:37,105] Trial 1 finished with value: 0.8230354986052387 and parameters: {'n_est': 15, 'max_depth': 8, 'drop': 0.15000000000000002, 'hidden_dim': 64, 'n_layers': 4, 'xgb_lr': 0.36000000000000004}. Best is trial 0 with value: 0.8675239039352517.


...

Early stopping
Fold 8 Accuracy: 0.8777777777777778 MCC: 0.755742181606458


Epochs:  32%|███▏      | 32/100 [00:01<00:03, 22.32it/s]


Early stopping
Fold 9 Accuracy: 0.9 MCC: 0.8017837257372732


Epochs:  28%|██▊       | 28/100 [00:01<00:03, 19.98it/s]


Early stopping
Fold 10 Accuracy: 0.8764044943820225 MCC: 0.7529056039533919
Elapsed time: 143.94 seconds


In [31]:
II_450_pcc75_results_df = train_with_best_hyperparameters(datasets, 'II_450_pcc75')

[I 2025-01-04 19:13:44,048] A new study created in memory with name: no-name-cd65c043-3f5a-44e5-b17a-fff6be8d778e


Currently finetuning DeepTLF model with dataset key: II_450_pcc75


Epochs:  32%|███▏      | 32/100 [00:01<00:03, 22.27it/s]
[I 2025-01-04 19:13:46,325] Trial 0 finished with value: 0.9120123092652646 and parameters: {'n_est': 15, 'max_depth': 8, 'drop': 0.1, 'hidden_dim': 128, 'n_layers': 6, 'xgb_lr': 0.31}. Best is trial 0 with value: 0.9120123092652646.


Early stopping


Epochs:  40%|████      | 40/100 [00:02<00:03, 16.86it/s]
[I 2025-01-04 19:13:49,812] Trial 1 finished with value: 0.9555555555555556 and parameters: {'n_est': 20, 'max_depth': 9, 'drop': 0.1, 'hidden_dim': 128, 'n_layers': 6, 'xgb_lr': 0.29000000000000004}. Best is trial 1 with value: 0.9555555555555556.


...

Early stopping
Fold 8 Accuracy: 0.8555555555555555 MCC: 0.7155417527999327


Epochs:  32%|███▏      | 32/100 [00:07<00:16,  4.14it/s]


Early stopping
Fold 9 Accuracy: 0.9111111111111111 MCC: 0.8254898842683464


Epochs:  34%|███▍      | 34/100 [00:08<00:16,  3.92it/s]


Early stopping
Fold 10 Accuracy: 0.898876404494382 MCC: 0.797979797979798
Elapsed time: 223.63 seconds


In [32]:
II_350_nopcc_results_df = train_with_best_hyperparameters(datasets, 'II_350_nopcc')

[I 2025-01-04 19:17:27,684] A new study created in memory with name: no-name-51c705ef-d6fe-4f94-84e0-b9642f93759a


Currently finetuning DeepTLF model with dataset key: II_350_nopcc


Epochs:  51%|█████     | 51/100 [00:02<00:02, 20.44it/s]
[I 2025-01-04 19:17:32,196] Trial 0 finished with value: 0.8603090020146066 and parameters: {'n_est': 30, 'max_depth': 3, 'drop': 0.30000000000000004, 'hidden_dim': 256, 'n_layers': 5, 'xgb_lr': 0.33}. Best is trial 0 with value: 0.8603090020146066.


Early stopping


Epochs:  30%|███       | 30/100 [00:00<00:01, 45.41it/s]
[I 2025-01-04 19:17:33,689] Trial 1 finished with value: 0.8603090020146066 and parameters: {'n_est': 10, 'max_depth': 3, 'drop': 0.25, 'hidden_dim': 384, 'n_layers': 2, 'xgb_lr': 0.38}. Best is trial 0 with value: 0.8603090020146066.


...

Early stopping


Fold 9 Accuracy: 0.9714285714285714 MCC: 0.944400281603035


Epochs:  40%|████      | 40/100 [00:13<00:19,  3.03it/s]

Early stopping


Fold 10 Accuracy: 0.8840579710144928 MCC: 0.7691684212097092
Elapsed time: 411.20 seconds


In [33]:
II_350_pcc95_results_df = train_with_best_hyperparameters(datasets, 'II_350_pcc95')

[I 2025-01-04 19:24:18,907] A new study created in memory with name: no-name-082dbd31-afd0-47ae-8309-c61bc2132d37


Currently finetuning DeepTLF model with dataset key: II_350_pcc95


Epochs:  55%|█████▌    | 55/100 [00:07<00:06,  7.15it/s]
[I 2025-01-04 19:24:29,021] Trial 0 finished with value: 0.8003267306650412 and parameters: {'n_est': 20, 'max_depth': 9, 'drop': 0.4, 'hidden_dim': 128, 'n_layers': 5, 'xgb_lr': 0.34}. Best is trial 0 with value: 0.8003267306650412.


Early stopping


Epochs:  39%|███▉      | 39/100 [00:02<00:03, 19.39it/s]
[I 2025-01-04 19:24:33,085] Trial 1 finished with value: 0.7431605356175383 and parameters: {'n_est': 50, 'max_depth': 3, 'drop': 0.30000000000000004, 'hidden_dim': 256, 'n_layers': 3, 'xgb_lr': 0.41000000000000003}. Best is trial 0 with value: 0.8003267306650412.


...

Early stopping
Fold 8 Accuracy: 0.9285714285714286 MCC: 0.8603090020146066


Epochs:  60%|██████    | 60/100 [00:03<00:02, 19.66it/s]


Early stopping
Fold 9 Accuracy: 1.0 MCC: 1.0


Epochs:  40%|████      | 40/100 [00:02<00:03, 18.39it/s]


Early stopping
Fold 10 Accuracy: 0.9420289855072463 MCC: 0.8840336134453781
Elapsed time: 152.39 seconds


In [34]:
II_350_pcc75_results_df = train_with_best_hyperparameters(datasets, 'II_350_pcc75')

[I 2025-01-04 19:26:51,307] A new study created in memory with name: no-name-0ca54d4f-2c50-4b5c-949e-77d65b5d7f38


Currently finetuning DeepTLF model with dataset key: II_350_pcc75


Epochs:  39%|███▉      | 39/100 [00:01<00:02, 25.33it/s]
[I 2025-01-04 19:26:53,457] Trial 0 finished with value: 0.8003267306650412 and parameters: {'n_est': 30, 'max_depth': 3, 'drop': 0.1, 'hidden_dim': 64, 'n_layers': 6, 'xgb_lr': 0.12}. Best is trial 0 with value: 0.8003267306650412.


Early stopping


Epochs:  34%|███▍      | 34/100 [00:02<00:04, 15.78it/s]
[I 2025-01-04 19:26:56,928] Trial 1 finished with value: 0.8574929257125441 and parameters: {'n_est': 20, 'max_depth': 10, 'drop': 0.25, 'hidden_dim': 320, 'n_layers': 2, 'xgb_lr': 0.43}. Best is trial 1 with value: 0.8574929257125441.


...

Early stopping
Fold 8 Accuracy: 0.9714285714285714 MCC: 0.944400281603035


Epochs:  49%|████▉     | 49/100 [00:02<00:03, 16.51it/s]


Early stopping
Fold 9 Accuracy: 1.0 MCC: 1.0


Epochs:  33%|███▎      | 33/100 [00:02<00:04, 15.22it/s]


Early stopping
Fold 10 Accuracy: 0.8840579710144928 MCC: 0.7691684212097092
Elapsed time: 90.72 seconds


In [35]:
print("Finetuned:")
print('\n---------------------------------------------------------------------\n')
view_results_df(II_all_nopcc_results_df, 'II_all_nopcc')
print('\n---------------------------------------------------------------------\n')
view_results_df(II_all_pcc95_results_df, 'II_all_pcc95')
print('\n---------------------------------------------------------------------\n')
view_results_df(II_all_nopcc_results_df, 'II_all_pcc75')
print('\n---------------------------------------------------------------------\n')
view_results_df(II_550_nopcc_results_df, 'II_550_nopcc')
print('\n---------------------------------------------------------------------\n')
view_results_df(II_550_pcc95_results_df, 'II_550_pcc95')
print('\n---------------------------------------------------------------------\n')
view_results_df(II_550_nopcc_results_df, 'II_550_pcc75')
print('\n---------------------------------------------------------------------\n')
view_results_df(II_450_nopcc_results_df, 'II_450_nopcc')
print('\n---------------------------------------------------------------------\n')
view_results_df(II_450_pcc95_results_df, 'II_450_pcc95')
print('\n---------------------------------------------------------------------\n')
view_results_df(II_450_nopcc_results_df, 'II_450_pcc75')
print('\n---------------------------------------------------------------------\n')
view_results_df(II_350_nopcc_results_df, 'II_350_nopcc')
print('\n---------------------------------------------------------------------\n')
view_results_df(II_350_pcc95_results_df, 'II_350_pcc95')
print('\n---------------------------------------------------------------------\n')
view_results_df(II_350_nopcc_results_df, 'II_350_pcc75')
print('\n---------------------------------------------------------------------\n')


Finetuned:

---------------------------------------------------------------------

10-fold cross validation results for results_df with key: II_all_nopcc
Fold 0, Accuracy: 0.858, MCC: 0.719
Fold 1, Accuracy: 0.816, MCC: 0.636
Fold 2, Accuracy: 0.861, MCC: 0.723
Fold 3, Accuracy: 0.849, MCC: 0.715
Fold 4, Accuracy: 0.800, MCC: 0.603
Fold 5, Accuracy: 0.812, MCC: 0.625
Fold 6, Accuracy: 0.857, MCC: 0.716
Fold 7, Accuracy: 0.820, MCC: 0.641
Fold 8, Accuracy: 0.849, MCC: 0.699
Fold 9, Accuracy: 0.837, MCC: 0.674

---------------------------------------------------------------------

10-fold cross validation results for results_df with key: II_all_pcc95
Fold 0, Accuracy: 0.829, MCC: 0.660
Fold 1, Accuracy: 0.849, MCC: 0.701
Fold 2, Accuracy: 0.910, MCC: 0.820
Fold 3, Accuracy: 0.845, MCC: 0.690
Fold 4, Accuracy: 0.812, MCC: 0.625
Fold 5, Accuracy: 0.865, MCC: 0.731
Fold 6, Accuracy: 0.841, MCC: 0.688
Fold 7, Accuracy: 0.902, MCC: 0.805
Fold 8, Accuracy: 0.853, MCC: 0.706
Fold 9, Accuracy: 0